# Visualizer Agent

This notebook implements the visualization component using Stable Diffusion for generating visual representations of data and concepts.

In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
import json
from typing import Dict, List
from utils.config import setup_logging, MODEL_CONFIGS, OUTPUTS_DIR
from utils.helpers import load_json
from tqdm import tqdm

In [ ]:
# Setup logging
logger = setup_logging('visualizer')

class VisualizationGenerator:
    def __init__(self):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model_id = MODEL_CONFIGS['visualization']['stable_diffusion_model']
        
        # Initialize Stable Diffusion
        self.pipeline = StableDiffusionPipeline.from_pretrained(
            self.model_id,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
        ).to(self.device)
    
    def generate_visualization(self, prompt: str, output_path: Path) -> Path:
        """Generate visualization using Stable Diffusion."""
        try:
            # Generate image
            image = self.pipeline(prompt).images[0]
            
            # Save image
            image.save(output_path)
            return output_path
            
        except Exception as e:
            logger.error(f"Error generating visualization: {str(e)}")
            return None

In [ ]:
def generate_visualizations() -> Dict:
    """Generate visualizations for hypotheses and experimental results."""
    logger.info('Starting visualization generation')
    
    # Create visualization directory
    vis_dir = OUTPUTS_DIR / 'visualizations'
    vis_dir.mkdir(exist_ok=True)
    
    # Load hypotheses and analysis results
    try:
        hypotheses = load_json(OUTPUTS_DIR / 'generated_hypotheses.json')
        analysis = load_json(OUTPUTS_DIR / 'experimental_analysis.json')
    except Exception as e:
        logger.error(f'Error loading results: {str(e)}')
        return {}
    
    # Initialize visualization generator
    generator = VisualizationGenerator()
    visualization_results = {}
    
    # Generate visualizations for hypotheses
    for title, data in tqdm(hypotheses.items(), desc='Generating hypothesis visualizations'):
        hypothesis = data['generated_hypothesis']
        prompt = f"Scientific visualization of hypothesis: {hypothesis}"
        
        output_path = vis_dir / f'hypothesis_{title}.png'
        if generator.generate_visualization(prompt, output_path):
            visualization_results[f'hypothesis_{title}'] = str(output_path)
    
    # Generate visualizations for experimental results
    for dataset, results in tqdm(analysis.items(), desc='Generating results visualizations'):
        # Create prompt based on feature importance
        feature_importance = results['feature_importance']
        prompt = f"Scientific data visualization showing relationship between {len(feature_importance)} features"
        
        output_path = vis_dir / f'results_{dataset}.png'
        if generator.generate_visualization(prompt, output_path):
            visualization_results[f'results_{dataset}'] = str(output_path)
    
    # Save visualization metadata
    metadata_path = vis_dir / 'visualization_metadata.json'
    with open(metadata_path, 'w') as f:
        json.dump(visualization_results, f, indent=2)
    
    logger.info(f'Saved visualization metadata to {metadata_path}')
    return visualization_results

In [ ]:
if __name__ == "__main__":
    # Generate visualizations
    results = generate_visualizations()
    
    # Print summary
    print(f"Generated {len(results)} visualizations")
    for name, path in results.items():
        print(f"\n{name}:")
        print(f"Saved to: {path}")